In [ ]:
## Awards 
querystr = '''
with psi_past_base_assets_title as 
    (select 
          title_id
        , coalesce(season_number,0) as season_number
        , title_name
        , content_category
        , program_type
        , category
        , tier
        , effective_start_date
        , effective_end_date
    from max_dev.workspace.psi_past_base_assets
    group by 1,2,3,4,5,6,7,8,9
    )
select 
      pba.*
      , itc.contributor_name
      , itc.name_id
      , itc.category as talent_category
      , itc.roles
      , itc.known_for
      , itc.credits_by_category
      , itc.listing_order
      , it.original_title
      , it.series_start_year
      , itcc.title_id as imdb_id_secondary
      , itcc.category as talent_category_secondary
      , itcc.roles as roles_secondary
      , itcc.listing_order as listing_order_secondary
      , ima.award_name
      , ima.award_category
      , winner
      , event
      , event_year
from psi_past_base_assets_title pba
left join max_prod.editorial.imdb_viewable_map ivm
    on pba.title_id = coalesce(ivm.viewable_id, ivm.viewable_series_id) 
left join max_prod.ckg.imdb_title_contributor itc
    on coalesce(ivm.imdb_id, ivm.imdb_series_id) = itc.title_id
left join max_prod.ckg.imdb_title_contributor itcc
    on itc.name_id = itcc.name_id
left join enterprise_data.catalog.imdb_title it
    on itcc.title_id = it.title_id    
inner join enterprise_data.catalog.imdb_movie_awards ima
    on itcc.title_id = ima.title_id    
where itc.category in ('director', 'writer','cast')
and itcc.category in ('director', 'writer','cast')
and itc.listing_order <=2
and itcc.listing_order<=2
and itcc.title_id!=itc.title_id
and ima.event_year::date < pba.effective_start_date::date
and event_year >= 2010
order by listing_order_secondary asc
'''

cursor_list = ctx.execute_string(
    querystr
    )
df_talent = pd.DataFrame.from_records(cursor_list[-1].fetchall(), columns=[x[0] for x in cursor_list[-1].description])
df_talent.columns= df_talent.columns.str.lower()
display(df_talent.head())

df_talent = df_talent.fillna(0)
print(df_talent.columns)

grpby = ['title_id', 'season_number', 'title_name', 'content_category',
       'program_type', 'category', 'tier', 'effective_start_date',
       'effective_end_date', 'name_id', 'contributor_name',
       'talent_category', 'listing_order'] #, 'language', 'wikipedia_site', 'wikipedia_article']
#          'imdb_id','known_for', 'credits_by_category',
#        'principal_cast_member', 'principal_crew_member', 
df_pv = df_talent.sort_values(by=grpby[:-3])
df_pv=df_pv.drop_duplicates(subset=grpby[:-3]+['award_name'], keep='first')

df_pv_t = df_pv[df_pv.winner=='true'][grpby+['award_name']].groupby(by=grpby).count().reset_index()

df_pv['event_year']=df_pv['event_year'].astype('int')
df_pv_t = df_pv[(df_pv.event_year>=2020)][grpby+['award_name']].groupby(by=grpby).count().reset_index()

df_filter = (df_pv_t.category!='Kids & Family') & (df_pv_t.talent_category=='writer')
df_pv_t[df_filter].sort_values(by='award_name', ascending=False).head(100)
df_pv_t=df_pv_t.drop_duplicates(subset=grpby[:12], keep='first')

df_pv_title = df_pv_t.groupby(by=['title_id','title_name','season_number','content_category','category','tier']).sum().reset_index()
# df_pv_title.loc[df_pv_title.tier>0, 'page_views']=1
# df_pv_title['page_views'] = np.log(df_pv_title['page_views'])
# df_pv_title.loc[df_pv_title.content_category=='series','page_views']=1